In [1]:
import sys
sys.path.append("C:\\Users\\user\\Desktop\\sesac_project\\NoF")
from Labor_precedent_EDA.db_info import user, password, host, port, database

In [2]:
# mysql.connector: MySQL 데이터베이스와 연결하기 위한 라이브러리
import mysql.connector

# pandas: 데이터 분석을 위한 라이브러리, 데이터프레임 생성 및 처리에 사용
import pandas as pd

# db_info 모듈에서 데이터베이스 연결 정보 가져오기
from Labor_precedent_EDA.db_info import user, password, host, database, port

# MySQL 데이터베이스에 연결
db_connection = mysql.connector.connect(
    host=host,          # 데이터베이스 호스트
    user=user,          # 사용자 이름
    password=password,  # 사용자 비밀번호
    database=database,  # 데이터베이스 이름
    port=port           # 포트 번호
        
)

# 데이터베이스 작업을 위한 커서 생성
cursor = db_connection.cursor()

# law_search_table에서 모든 데이터 선택
cursor.execute("SELECT * FROM labor_law_ref")

# 쿼리 실행 결과를 가져옴
result = cursor.fetchall()

# 결과를 데이터프레임으로 변환, 컬럼 이름은 쿼리 결과의 첫 번째 행에서 가져옴
labor_law_ref = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

# law_search 데이터프레임 출력
labor_law_ref

,참조조문,split_text
0,"[1] 근로기준법 제28조, 제29조, 노동위원회법 제23조 / [2] 근로기준법...","근로기준법 제28조, 제29조, 노동위원회법 제23조, 근로기준법 제2조 제1항 제..."
1,"[1] 파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민법 제390조, 제3...","파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민법 제390조, 제393조, ..."
2,구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 개...,구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 개...
3,[1] 파견근로자 보호 등에 관한 법률 제2조 제1호 / [2] 파견근로자 보호 ...,"파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제..."
4,"[1] 근로기준법 제18조 / [2] 근로기준법 제18조, 구 고등교육법 시행령(...","근로기준법 제18조, 근로기준법 제18조, 구 고등교육법 시행령(2024. 2. 2..."
...,...,...
5011,소득세법(구) 제32조<br/>,소득세법(구) 제32조
5012,"근로기준법 제28조\n, \n \n \n 제9...","근로기준법 제28조\n, \n \n \n 제9..."
5013,"행정소송법(법률 제213호) 제12조, \n 귀속재산처리법 제15조<br/>","행정소송법(법률 제213호) 제12조, \n 귀속재산처리법 제15조"
5014,"노동조합법 제33조, 제34조, 노동위원회규칙 제5조<br/>","노동조합법 제33조, 제34조, 노동위원회규칙 제5조"


# 1차 시도 : 패턴추출

In [3]:
from sklearn.cluster import DBSCAN
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd

# KoSentenceBERT 모델 로드 (한국어 지원)
model_name = "jhgan/ko-sbert-sts"  # 한국어 문장 유사도 분석을 위한 KoSentenceBERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 법조항 데이터 예시
sentences = list(labor_law_ref['split_text'].iloc[:499])


# 문장 임베딩 함수 정의
def get_embeddings(sentences):
    # 문장들을 토큰화하고 텐서 형태로 변환
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # 문장 임베딩 평균값 사용
    return embeddings.numpy()

# 문장 임베딩 생성
embeddings = get_embeddings(sentences)

# DBSCAN을 이용한 클러스터링 (eps와 min_samples는 데이터에 따라 조정)
clustering = DBSCAN(eps=0.5, min_samples=2, metric="cosine").fit(embeddings)

# 결과를 데이터프레임에 저장
df = pd.DataFrame({"문장": sentences, "클러스터 라벨": clustering.labels_})

# 클러스터별 대표 문장과 패턴 출력
for label in set(clustering.labels_):
    cluster_sentences = df[df["클러스터 라벨"] == label]["문장"].values
    if label == -1:
        print("\n새로운 패턴으로 인식된 문장들:")
    else:
        print(f"\n클러스터 {label}에 속한 문장들:")
    for sentence in cluster_sentences:
        print(f" - {sentence}")

# CSV 파일로 저장 (선택 사항)
df.to_csv("clustered_patterns.csv", index=False, encoding="utf-8-sig")


클러스터 0에 속한 문장들:
 - 근로기준법 제28조, 제29조, 노동위원회법 제23조, 근로기준법 제2조 제1항 제1호, 근로기준법 제2조 제1항 제1호, 제28조
 - 파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민법 제390조, 제393조, 파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민사소송법 제248조[소의 이익], 민법 제162조 제1항
 - 구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 개정되기 전의 것) 제6조 제3항(현행 제6조의2 제1항 제3호, 제2항 참조), 민법 제413조
 - 파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민사소송법 제248조[소의 이익], 민법 제162조 제1항
 - 근로기준법 제18조, 근로기준법 제18조, 구 고등교육법 시행령(2024. 2. 20. 대통령령 제34227호로 개정되기 전의 것) 제6조 제1항
 - 구 건설산업기본법(2018. 12. 31. 법률 제16136호로 개정되기 전의 것) 제25조, 제96조 제4호, 구 근로기준법(2019. 4. 30. 법률 제16415호로 개정되기 전의 것) 제44조의2, 제109조, 구 근로기준법(2019. 4. 30. 법률 제16415호로 개정되기 전의 것) 제44조의2, 제109조
 - 근로기준법 제1조, 제2조 제1항 제1호, 제17조, 제114조 제1호, 기간제 및 단시간근로자 보호 등에 관한 법률 제1조, 제17조, 제24조 제2항 제2호
 - 민법 제393조, 제763조, 민법 제393조, 제763조, 근로기준법 제50조 제1항, 관공서의 공휴일에 관한 규정 제3조, 제4조
 - 근로기준법 제2조 제1항 제2호, 근로기준법 제111조
 - 민법 제538조, 근로기준법 제46조
 - 파견근로자 보호 등에 관한 법률 제2조 제1호, 민법 제393조, 제763조, 구 파견근로자보호 등에 관한 법률(2012. 2. 1

* 패턴 추출 실패

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   문장       499 non-null    object
 1   클러스터 라벨  499 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [12]:
print(df[df['클러스터 라벨']== 1])

          문장  클러스터 라벨
13   참조조문 없음        1
15   참조조문 없음        1
43   참조조문 없음        1
45   참조조문 없음        1
69   참조조문 없음        1
..       ...      ...
360  참조조문 없음        1
364  참조조문 없음        1
369  참조조문 없음        1
372  참조조문 없음        1
378  참조조문 없음        1

[114 rows x 2 columns]


In [5]:
df['클러스터 라벨'].value_counts()

클러스터 라벨
0    383
1    114
2      2
Name: count, dtype: int64

In [13]:
label_value = 2
filtered_data = df[df['클러스터 라벨'] == label_value]
filtered_data

,문장,클러스터 라벨
42,,2
85,,2


# 2차 시도 : 패턴 추출 20개로 갯수 지정하여 진행

In [34]:
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd

# KoSentenceBERT 모델 로드 (한국어 지원)
model_name = "jhgan/ko-sbert-sts"  # 한국어 문장 유사도 분석을 위한 KoSentenceBERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 법조항 데이터 예시
sentences = list(law_pattns1['조문_분리2'].iloc[:1000])  # labor_law_ref 데이터프레임에 맞게 변경

# 문장 임베딩 함수 정의
def get_embeddings(sentences):
    # 문장들을 토큰화하고 텐서 형태로 변환
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # 문장 임베딩 평균값 사용
    return embeddings.numpy()

# 문장 임베딩 생성
embeddings = get_embeddings(sentences)

# KMeans를 이용한 20개 클러스터로 클러스터링
kmeans = KMeans(n_clusters=20, random_state=0).fit(embeddings)

# 결과를 데이터프레임에 저장
df = pd.DataFrame({"문장": sentences, "클러스터 라벨": kmeans.labels_})

# 클러스터별 대표 문장과 패턴 출력
for label in range(30):  # 20개의 클러스터
    cluster_sentences = df[df["클러스터 라벨"] == label]["문장"].values
    print(f"\n클러스터 {label}에 속한 문장들:")
    for sentence in cluster_sentences[:5]:  # 대표 문장 5개만 출력
        print(f" - {sentence}")

# CSV 파일로 저장
df.to_csv("clustered_patterns.csv", index=False, encoding="utf-8-sig")


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [24]:
label_value = 10
filtered_data = df[df['클러스터 라벨'] == label_value]
filtered_data

,문장,클러스터 라벨
23,"헌법 제11조 제1항, 근로기준법 제6조, 민법 제750조",10
26,"근로기준법 제2조 제1항 제1호, 제17조, 제26조, 제36조, 근로기준법 제2조...",10
39,"근로기준법 제2조 제1항 제2호, 제17조, 제36조, 제109조 제1항, 제114...",10
191,"근로기준법 제36조, 제109조 제1항, 근로기준법 제15조, 제36조, 제109조...",10
206,"근로기준법 제2조 제1항 제1호, \n 제4호, 근로기준법 제36조, ...",10
248,"근로기준법 제26조, \n 제110조 제1호, 근로기준법 제26조, \...",10
305,"근로기준법 제15조, \n 제36조, \n 제112조, 근...",10
311,"근로기준법 제24조, \n 제115조, \n 근로기준법 시...",10
312,"근로기준법 제30조 제1항, 제110조, 근로기준법 제30조 제1항,...",10
315,"근로기준법 제36조, \n 제112조, 근로기준법 제36조, \n ...",10


In [27]:
df['문장'].loc[481]

'가.\n \n        근로기준법 제109조, \n\n \n        제36조, 나. \n        제110조, \n\n \n        제27조의2'

In [25]:
df['클러스터 라벨'].value_counts()

클러스터 라벨
1     114
0      40
14     40
9      32
8      29
18     27
5      26
10     24
2      23
15     18
4      18
11     18
13     17
17     16
3      15
7      14
12     11
19      9
6       6
16      2
Name: count, dtype: int64

In [ ]:
law_pattns1 = pd.read_csv(r'C:\Users\user\Desktop\sesac_project\NoF\Labor_precedent_EDA\law_pattn.csv')

In [36]:
law_pattns1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사건번호    5016 non-null   object
 1   참조문     5004 non-null   object
 2   참조판례    5003 non-null   object
 3   조문_분리   5016 non-null   object
 4   조문_분리2  5004 non-null   object
 5   패턴1     3638 non-null   object
dtypes: object(6)
memory usage: 235.3+ KB


In [37]:
type(law_pattns1)

pandas.core.frame.DataFrame

In [39]:
law_pattns1=law_pattns1.fillna("")

In [40]:
law_pattns1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사건번호    5016 non-null   object
 1   참조문     5016 non-null   object
 2   참조판례    5016 non-null   object
 3   조문_분리   5016 non-null   object
 4   조문_분리2  5016 non-null   object
 5   패턴1     5016 non-null   object
dtypes: object(6)
memory usage: 235.3+ KB


In [ ]:
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd

# KoSentenceBERT 모델 로드 (한국어 지원)
model_name = "jhgan/ko-sbert-sts"  # 한국어 문장 유사도 분석을 위한 KoSentenceBERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 법조항 데이터 예시 - None 값 제거
sentences = list(law_pattns1['조문_분리2'].iloc[:1500])

# 문장 임베딩 함수 정의
def get_embeddings(sentences):
    # 문장들을 개별적으로 토큰화하고 텐서 형태로 변환
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # 문장 임베딩 평균값 사용
    return embeddings.numpy()

# 문장 임베딩 생성
embeddings = get_embeddings(sentences)

# KMeans를 이용한 20개 클러스터로 클러스터링
kmeans = KMeans(n_clusters=20, random_state=0).fit(embeddings)

# 결과를 데이터프레임에 저장
df = pd.DataFrame({"문장": sentences, "클러스터 라벨": kmeans.labels_})

# 클러스터별 대표 문장과 패턴 출력
for label in range(30):  # 30개의 클러스터
    cluster_sentences = df[df["클러스터 라벨"] == label]["문장"].values
    print(f"\n클러스터 {label}에 속한 문장들:")
    for sentence in cluster_sentences[:5]:  # 대표 문장 5개만 출력
        print(f" - {sentence}")

# CSV 파일로 저장
df.to_csv("clustered_patterns.csv", index=False, encoding="utf-8-sig")


In [43]:
df['클러스터 라벨'].value_counts()

클러스터 라벨
0     252
11     83
5      76
16     70
2      68
6      54
1      46
4      44
7      42
13     39
10     31
14     28
3      28
19     24
9      23
8      23
18     22
17     22
12     16
15      9
Name: count, dtype: int64

In [44]:
label_value = 11
filtered_data = df[df['클러스터 라벨'] == label_value]
filtered_data

,문장,클러스터 라벨
11,민사소송법 제250조/구 파견근로자 보호 등에 관한 법률(2006. 12. 21. ...,11
12,구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호로 개정...,11
17,구 근로기준법(2017. 11. 28. 법률 제15108호로 개정되기 전의 것) 제...,11
20,구 산업안전보건법(2019. 1. 15. 법률 제16272호로 전부 개정되기 전의 ...,11
27,"근로기준법 제2조 제1항 제1호/근로기준법 제2조 제1항 제1호, 제2호/근로기준법...",11
...,...,...
951,구 산업재해보상보험법(2018. 6. 12. 법률 제15665호로 개정되기 전의 것...,11
956,"근로기준법 제60조, 구 노인장기요양보험법(2018. 12. 11. 법률 제1588...",11
959,"헌법 제32조 제4항, 제36조 제2항, 구 고용보험법(2002. 12. 30. 법...",11
967,구 근로기준법(2018. 3. 20. 법률 제15513호로 개정되기 전의 것) 제2...,11


In [ ]:
# 각 클러스터 중심 확인
cluster_centers = kmeans.cluster_centers_

# 클러스터 중심과 각 데이터 포인트의 거리 계산
df["클러스터 중심까지 거리"] = [
    np.linalg.norm(embedding - cluster_centers[label]) for embedding, label in zip(embeddings, kmeans.labels_)
]

# 클러스터별 중심 좌표 및 평균 거리 출력
for label in range(20):
    cluster_data = df[df["클러스터 라벨"] == label]
    mean_distance = cluster_data["클러스터 중심까지 거리"].mean()
    print(f"\n클러스터 {label}:")
    print(f" - 중심 좌표: {cluster_centers[label]}")
    print(f" - 중심으로부터 평균 거리: {mean_distance}")
    print(f" - 클러스터에 속한 문장 개수: {len(cluster_data)}")